## Check dicom and mhd
---

### simple Reading / checking format

In [ ]:
import pydicom
import SimpleITK

d = pydicom.read_file('./Dicom/L0/SE0/IM0')
m = SimpleITK.ReadImage('./mhd/S2.mhd')

# print(d)
print("Dicom Maker >> ", d.Manufacturer)
print("Dicom Shape >> ", d.pixel_array.shape)

# print(m)
print("mhd Shape >>", SimpleITK.GetArrayFromImage(m).shape)

### dicom Reading(continuous) / checking lack

In [ ]:
import glob
from natsort import natsorted
import numpy as np
import traceback

# 同じディレクトリにあるdicomファイルを連続したデータに変換する
def Read_dicom_boxel(path):
    d_names = []
    d_names.extend(reversed(natsorted(glob.glob(path+"*"))))

    ref_dicom = pydicom.read_file(d_names[0])
    d_array   = np.zeros((ref_dicom.Rows, ref_dicom.Columns, len(d_names)), dtype=ref_dicom.pixel_array.dtype)
    
    for d_name in d_names:
        try:
            d = pydicom.read_file(d_name)
            d_array[:, :, d_names.index(d_name)] = d.pixel_array
        except: 
            #traceback.print_exc()
            print("err >> ", d_name)
            
    print('_'*5,'Read %s'%path,'_'*2, d_array.shape, '_'*5)
    return d_array


# 上から順に dicom mhd 教師データ
d_array = Read_dicom_boxel('./Dicom/L0/SE0/')
m_array = SimpleITK.GetArrayFromImage(m)                                       # ↓ 0が途切れ途切れ, 1が繋がっている
t_array = (SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(natsorted(glob.glob('./GroundTruth/L0/*.mhd'))[0])))*255

### Display dicom and mhd

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 20), dpi=25)
plt.rcParams['figure.figsize'] = (15.0, 5.0)

@interact(slices=(0, d_array.shape[2]-1))
def plot_rolling_mean(slices=143):
    plt.subplot(131)
    plt.imshow(d_array[:, :, slices])
    plt.title('dicom data')
    
    plt.subplot(132)
    plt.imshow(m_array[slices, :, :])
    plt.title('mhd data')
    
    plt.subplot(133)
    plt.imshow(t_array[slices, :, :])
    plt.title('ground truth')
    
#     plt.gray()
#     plt.show()


### display adipose

In [ ]:
import cv2
import copy

plt.rcParams['figure.figsize'] = (15.0, 5.0)
mini = np.min(m_array)
maxi = np.max(m_array)

@interact(slices=(0, m_array.shape[0]-1), minimum=(mini, maxi),maximum=(mini, maxi))
def plot_adipose(slices,minimum=-1900,maximum=2000):
    
    d_adipose = copy.deepcopy(d_array)             # 論文内だと下記の範囲が脂肪(元は-190~-30)
    d_adipose[d_adipose < minimum] = 0          #  ---- | -250 | +++++++++++++++++++
    d_adipose[d_adipose > maximum] = 0          #  +++++++++++++++++++++| -30 | ----
    
    m_adipose = copy.deepcopy(m_array)             # 論文内だと下記の範囲が脂肪(元は-190~-30)
    m_adipose[m_adipose < minimum] = 0          #  ---- | -250 | +++++++++++++++++++
    m_adipose[m_adipose > maximum] = 0          #  +++++++++++++++++++++| -30 | ----
    
    plt.subplot(131)
    plt.imshow((d_adipose[ :, :, 163]))
    plt.title('dicom data')
    
    plt.subplot(132)
    plt.imshow(cv2.bitwise_not(m_adipose[163, :, : ]))
    plt.title('mhd data')
    
    plt.subplot(133)
    plt.imshow(t_array[163, :, :])
    plt.title('ground truth')

    #cv2.bitwise_not

## Make train/test folder for U-Net

In [ ]:
import os
import shutil


normal_mhd_dir = './mhd/*.mhd'
GT_mhd_dir   = './GroundTruth/*/*[!2].mhd'
UNet_mhd_dir = './U-Net_DataSet/'

if not os.path.exists(UNet_mhd_dir): 
    os.makedirs(UNet_mhd_dir+'origin')
    os.makedirs(UNet_mhd_dir+'train/input')
    os.makedirs(UNet_mhd_dir+'train/target')
    os.makedirs(UNet_mhd_dir+'test/input')
    os.makedirs(UNet_mhd_dir+'test/target')

# UNet_mhd_dir の originディレクトリに集約
for normal_mhd in natsorted(glob.glob(normal_mhd_dir)):
    shutil.copyfile(normal_mhd, UNet_mhd_dir+'origin/'+normal_mhd.split('/')[-1])
for GT_mhd in natsorted(glob.glob(GT_mhd_dir)):
    shutil.copyfile(GT_mhd, UNet_mhd_dir+'origin/'+GT_mhd.split('/')[-2]+'_GT.mhd')

# 集約したoriginファイルから探索
mhd_list = natsorted(glob.glob(UNet_mhd_dir+'origin/*_GT.mhd'))
train_percentage = 0.8

for origin_GT_mhd in mhd_list[: int(len(mhd_list)*train_percentage)]:
    shutil.copyfile(origin_GT_mhd, UNet_mhd_dir+'train/target/' + origin_GT_mhd.split('/')[-1])
    shutil.copyfile(origin_GT_mhd.replace('_GT',''), UNet_mhd_dir+'train/input/' + origin_GT_mhd.split('/')[-1].replace('_GT',''))

for origin_GT_mhd in mhd_list[int(len(mhd_list)*train_percentage) :]:
    shutil.copyfile(origin_GT_mhd, UNet_mhd_dir+'test/target/' + origin_GT_mhd.split('/')[-1])
    shutil.copyfile(origin_GT_mhd.replace('_GT',''), UNet_mhd_dir+'test/input/' + origin_GT_mhd.split('/')[-1].replace('_GT',''))

## ----------------------
↓ this is for pix2pix / case of using png image at input

## Read continuous slices
---

In [ ]:
import os

def Canny_filter(use_array):
    # Cannyを使う際にdatatypeに少し注意 -> hconcatでも同じdatatype必要に
    return cv2.Canny(np.uint8(use_array),10,100)

def Laplacian_filter(use_array):
    # Laplacianはノイズがひどい
    return np.uint8(cv2.Laplacian(np.uint8(use_array), cv2.CV_32F))

def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = SimpleITK.ReadImage(filename)
    # 0-1なので255倍に / Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = (SimpleITK.GetArrayFromImage(itkimage))*255
    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    #origin = np.array(list(reversed(itkimage.GetOrigin())))
    # Read the spacing along each dimension
    #spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return ct_scan


def Save_dicom2image(d_array):
    # make directory
    save_dir = dir_name.replace('Dicom','png')
    gt_path  = dir_name.replace('Dicom','GroundTruth')+'/*.mhd'
    
    # adjust dicom value(-2000~3000) to png value(0~256)
    d_array = d_array - np.min(d_array)
    d_array = (d_array / np.max(d_array) *256).astype('int64')
 
    try:
        if not os.path.exists(save_dir): os.makedirs(save_dir)
        
        gt_array = load_itk(natsorted(glob.glob(gt_path))[1])
        for slices in range(d_array.shape[2]):
            imgs = cv2.hconcat([ np.uint8(d_array[:,:,slices]), np.uint8(gt_array[slices,:,:]) ])
            cv2.imwrite(save_dir+'/'+str(slices)+'.png', cv2.cvtColor(imgs, cv2.COLOR_GRAY2BGR))
    except:                    
        #traceback.print_exc()
        print("err >> ", gt_path)
        

In [ ]:
dicom_dir = './Dicom/*'
gt_dir = './GroundTruth/*/*.mhd'


for dir_name in natsorted(glob.glob(dicom_dir)):
    Save_dicom2image(Read_dicom_boxel(dir_name+"/*/"))

## Make Train / Val directory

In [ ]:
import shutil

root_dir = './'

if not os.path.exists(root_dir+'train'): os.makedirs(root_dir+'train')
if not os.path.exists(root_dir+'val'): os.makedirs(root_dir+'val')

for png_file_path in natsorted(glob.glob(root_dir+'png/*'))[:12]:
    for png_img_path in natsorted(glob.glob(png_file_path+"/*")):
        each_dir = png_img_path.split('/')
        shutil.copyfile(png_img_path, root_dir+'train/'+each_dir[-2]+'-'+each_dir[-1])
        
for png_file_path in natsorted(glob.glob(root_dir+'png/*'))[12:]:
    for png_img_path in natsorted(glob.glob(png_file_path+"/*")):
        each_dir = png_img_path.split('/')
        shutil.copyfile(png_img_path, root_dir+'val/'+each_dir[-2]+'-'+each_dir[-1])

## 今後すること
---

精度検証

データ拡張(2万枚くらいに)
- コントラスト変化
- ノイズ追加
- 



In [ ]:
import glob, SimpleITK
from natsort import natsorted

mhd_files = natsorted(glob.glob('../Preprocess/GroundTruth/'+'*/*[!2].mhd'))
images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
image_each_length = images.shape[0]

for mhd_name in mhd_files:
    mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
    images = np.append(images,mhd_array,axis=0)
    image_each_length = np.append(image_each_length,mhd_array.shape[0])

print(images.shape, image_each_length)

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt


plt.figure(figsize=(20, 20), dpi=25)
plt.rcParams['figure.figsize'] = (15.0, 10.0)

@interact(slices=(0, max(image_each_length)), case1=(0,len(image_each_length)-2), case2=(0,len(image_each_length)-2))
def plot_rolling_mean(slices,case1=0,case2=10):
    
    case1_slice = images[np.sum(image_each_length[:case1]):np.sum(image_each_length[:case1])+image_each_length[case1]]
    case2_slice = images[np.sum(image_each_length[:case2]):np.sum(image_each_length[:case2])+image_each_length[case2]]

    plt.subplot(121)
    plt.imshow(case1_slice[ slices,:, : ])
    plt.title('#'+str(case1)+"  "+mhd_files[case1].replace("../Preprocess/GroundTruth/",""))
    
    plt.subplot(122)
    plt.imshow(case2_slice[ slices, :, :])
    plt.title('#'+str(case2)+"  "+mhd_files[case2].replace("../Preprocess/GroundTruth/",""))
    
#     plt.gray()
#     plt.show()
